In [4]:
from torch import nn
from torch.nn import functional as F
import torch

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
    

In [5]:
net = MLP()
X = torch.rand(2, 20)
net(X)

tensor([[ 0.1539,  0.0323, -0.0991,  0.0471,  0.0399, -0.0063,  0.0506,  0.0594,
         -0.1306, -0.1633],
        [ 0.2420, -0.0696, -0.1109,  0.0181,  0.0148, -0.0228,  0.1685,  0.0066,
         -0.1750, -0.1351]], grad_fn=<AddmmBackward0>)

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1682,  0.0739,  0.1177,  0.1058,  0.3374, -0.2328, -0.0016, -0.0855,
          0.3177, -0.0748],
        [ 0.1628,  0.0322,  0.0316,  0.1410,  0.2387, -0.2015,  0.0305, -0.1340,
          0.3316, -0.0119]], grad_fn=<AddmmBackward0>)

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [9]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0467, grad_fn=<SumBackward0>)

In [10]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))

In [11]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1347, grad_fn=<SumBackward0>)